In [ ]:
!pip install https://github.com/facebookresearch/fastText/archive/master.zip

In [2]:
from deeppavlov.models.classifiers.keras_cnn_classifier import keras_cnn_classifier
from tensorflow.python.keras._impl.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [3]:
categories = [  # choose only several categories or left blank for loading all 20 categories
    "alt.atheism",
    "sci.space"
]
train_data = fetch_20newsgroups(subset='train', categories=categories)
test_data = fetch_20newsgroups(subset='test', categories=categories)

In [4]:
print("Example of label from dataset: ", train_data.target[1050])
print("Number of training instances: ", len(train_data.data))
print("Number of classes: ", len(train_data.target_names))
print("List of labels: ", train_data.target_names)
print()
print("Example of instance data: ", train_data.data[1050])


Example of label from dataset:  1
Number of training instances:  1073
Number of classes:  2
List of labels:  ['alt.atheism', 'sci.space']

Example of instance data:  From: John Lussmyer <dragon@angus.mi.org>
Subject: Re: DC-X update???
Organization: Mystery Spot BBS
Reply-To: dragon@angus.mi.org
Lines: 12

henry@zoo.toronto.edu (Henry Spencer) writes:

> The first flight will be a low hover that will demonstrate a vertical
> landing.  There will be no payload.  DC-X will never carry any kind

Exactly when will the hover test be done, and will any of the TV
networks carry it.  I really want to see that...

--
John Lussmyer (dragon@angus.mi.org)
Mystery Spot BBS, Royal Oak, MI --------------------------------------------?--




In [6]:
import fastText
embedder = fastText.load_model('DeepPavlov/download/embeddings/dstc2_fastText_model.bin')

In [7]:
train_vectors = [np.sum([embedder.get_sentence_vector(line) for line in x.split()], axis=0) for x in train_data.data]
test_vectors = [np.sum([embedder.get_sentence_vector(line) for line in x.split()], axis=0) for x in test_data.data]

In [8]:
len(train_vectors[0])

100

In [9]:
mlp_binary_clf = MLPClassifier()


In [10]:
mlp_binary_clf.fit(train_vectors, train_data.target)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [11]:
mlp_binary_clf.predict([embedder.get_sentence_vector("scientific text about space")])

array([1])

In [12]:
mlp_binary_clf.score(test_vectors, test_data.target)

0.5301542776998598

In [13]:
keras_params = {"n_classes": 2,
                "text_size": 1,  # 1 for single text embedding
                "embedding_size": len(train_vectors[0])}
sk_params = {"batch_size": 16,
             "epochs": 3}
cnn_binary_clf = KerasClassifier(build_fn=keras_cnn_classifier, build_fn_params=keras_params, **sk_params)

In [14]:
cnn_binary_clf.fit(np.array(train_vectors)[:, np.newaxis], train_data.target)

Epoch 1/3


  16/1073 [..............................] - ETA: 26s - loss: 6.0394 - acc: 0.6250 - binary_accuracy: 0.6250 - fmeasure: 0.7692

 240/1073 [=====>........................] - ETA: 1s - loss: 7.1887 - acc: 0.5500 - binary_accuracy: 0.5500 - fmeasure: 0.6942 

 448/1073 [===========>..................] - ETA: 0s - loss: 6.9832 - acc: 0.5625 - binary_accuracy: 0.5625 - fmeasure: 0.7083

 672/1073 [=================>............] - ETA: 0s - loss: 7.0754 - acc: 0.5565 - binary_accuracy: 0.5565 - fmeasure: 0.7039

...

 240/1073 [=====>........................] - ETA: 0s - loss: 7.1076 - acc: 0.5542 - binary_accuracy: 0.5542 - fmeasure: 0.7093

 496/1073 [============>.................] - ETA: 0s - loss: 7.3926 - acc: 0.5363 - binary_accuracy: 0.5363 - fmeasure: 0.6932

 752/1073 [====================>.........] - ETA: 0s - loss: 7.0808 - acc: 0.5559 - binary_accuracy: 0.5559 - fmeasure: 0.7088

1008/1073 [===========================>..] - ETA: 0s - loss: 7.0381 - acc: 0.5585 - binary_accuracy: 0.5585 - fmeasure: 0.7113

1073/1073 [==============================] - 0s 210us/step - loss: 7.1317 - acc: 0.5527 - binary_accuracy: 0.5527 - fmeasure: 0.7058


In [15]:
cnn_binary_clf.score(np.array(test_vectors)[:, np.newaxis], test_data.target)

 16/713 [..............................] - ETA: 2s

713/713 [==============================] - 0s 133us/step


0.5525946704485306

In [20]:
cnn_binary_clf.predict(np.array([embedder.get_sentence_vector("scientific text about space")])[:, np.newaxis])


AttributeError: 'Model' object has no attribute 'predict_classes'

In [ ]:
pipeline = Pipeline([
    ('emb', FastTextEmbedder()),
    ('clf', my_model),
])
